In [6]:
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import mplfinance as mpl
import numpy as np
import plotly.graph_objects as go
from plotly import tools

#install symbols nasdaq100の銘柄コード
df = pd.read_csv('../symbols/nasdaq100.csv')

# sign = []
# for symbol in df['symbol']:
#     sign.append(symbol)
    

sign = ['6976.T','4477.T','6036.T']
    

totalresult = []
start = '2019-01-01'
end = '2021-02-02'
    
'''
signは銘柄.銘柄ごとにfor分を回している.
'''

for i in sign:
    stock_code = i 
    df = yf.download(stock_code, start, end, interval='1d') 
    
    
    print(i)
    term=81
    # chek the highest price in the past {term} times
    df['Highest'+str(term)] = df.iloc[:, 4].rolling(window=term).max()
    print(df.head())
    
        
    
    # calculation SMA
    short_sma = 25
    long_sma = 50
    verylong_sma = 250
    #'term'日間の移動平均線
    df['SMA'+str(short_sma)] = df['Adj Close'].rolling(window=short_sma).mean()
    df['SMA'+str(long_sma)] = df['Adj Close'].rolling(window=long_sma).mean()
    df['SMA'+str(verylong_sma)] = df['Adj Close'].rolling(window=verylong_sma).mean()
      
    #slope of the sma SMAの傾き
    def diff(x):
        return x[-1] - x[0]
    trend_term = 2
    df['SMA Trend'] = df['SMA'+str(short_sma)].rolling(window=trend_term).apply(diff)
    
     # 損切り率
    stopLoss = 0.94
 
    
    #買い日と売り日とシグナルが出た日
    buy_days = []
    sell_days = []
    signal_days = []
    
    #holding period list 保有日数のリストで平均保有日数を出したい
    holding_period_list = []


    #calucurate Bband
    df['MA'] = df['Adj Close'].rolling(window=20).mean()
    df['STD'] = df['Adj Close'].rolling(window=20).std()
    df['Upper'] = df['MA'] + (df['STD'] * 2)
    df['Lower'] = df['MA'] - (df['STD'] * 2)
    
    #daviation_rate 移動平均線乖離率
    df['daviation_rate'] = 100 * (df['Adj Close'] - df['SMA'+str(short_sma)]) / df['SMA'+str(short_sma)]

    #Standard deviation normalization 正規化してみた
    df['Standard_deviation_normalization'] = 100 * 2 * df['STD'] / df['SMA'+str(short_sma)]
   
    buy_position = False
    isSignal = False
    percentChange = []
    for i in range(1, len(df)):
        close = df['Adj Close'][i]
        STD = df['STD'][i]
        high_price = df['High'][i]
        low_price = df['Low'][i]
        open_price = df['Open'][i]
        sma = df['SMA'+str(short_sma)][i]
        SDN = df['Standard_deviation_normalization'][i]
        daviation_rate = df['daviation_rate'][i]
        Highest = df['Highest'+str(term)][i-1] #過去term日間の最高値
        

        # シグナルが出た日を示している
        if np.isnan(df['MA'][i]) == False:
            if buy_position == False and SDN < 10 and -5 < daviation_rate < 5 :
#                 print('low volatility')
                isSignal = True
                signalday = pd.to_datetime(df.index[i])
                signal_days.append(i)
                print(signalday)
            
        if isSignal == True and (pd.to_datetime(i) - signalday).days < 7 and close > Highest and buy_position == False:
            buy_position = True
            buyday = pd.to_datetime(df.index[i])
            buy_days.append(buyday)
            buy_price = close
            buy_stopLine = close * stopLoss
            buypoint = i
            current_stop_level = 0
            isSignal = False
            print('buy in {} at {}'.format(buyday,buy_price))
           
        
        #adjustment stopline
        if buy_position == True and close > buy_price * 1.05 and current_stop_level == 0:
            buy_stopLine = buy_price
            current_stop_level = 1
            print('up stopline1 in {} at {}'.format(df.index[i],buy_stopLine))
            
        if buy_position == True and close > buy_price * 1.1 and current_stop_level == 1:
            buy_stopLine = buy_price * 1.05
            current_stop_level = 2
            print('up stopline2 in {} at {}'.format(df.index[i],buy_stopLine))
            
        if buy_position == True and close > buy_price * 1.2 and current_stop_level == 2:
            buy_stopLine = buy_price * 1.1
            current_stop_level = 3
            print('up stopline3 in {} at {}'.format(df.index[i],buy_stopLine))
            
        if buy_position == True and close > buy_price * 1.3 and current_stop_level == 3:
            buy_stopLine = close * 0.85
            current_stop_level = 4
            print('up stopline4 in {} at {}'.format(df.index[i],buy_stopLine))
            
        if buy_position == True and close > buy_price * 1.5 and current_stop_level == 4:
            buy_stopLine = close * 0.8
            current_stop_level = 5  
            print('up stopline5 in {} at {}'.format(df.index[i],buy_stopLine))      
            
        if buy_position == True and close > Highest and current_stop_level == 5:
            buy_stopLine = close * 0.8
            current_stop_level = 5  
            print('up stopline6 in {} at {}'.format(df.index[i],buy_stopLine))
            
            
            
        # 損切り or Profit taking
        # 終値始値が損切りラインを下回ったら売る。
        if buy_position == True and open_price < buy_stopLine and i != buypoint:
            buy_position = False 
            percent = (open_price/buy_price - 1) * 100
            percentChange.append(percent)
            sellday = pd.to_datetime(df.index[i])
            sell_days.append(sellday)
            holding_period = sellday - buyday
            print('sell in {} at {} | profit_one is {}'.format(sellday,open_price,percent))
            
        elif buy_position == True and close < buy_stopLine and i != buypoint:
            buy_position = False 
            percent = (buy_stopLine/buy_price - 1) * 100
            percentChange.append(percent)
            sellday = pd.to_datetime(df.index[i])
            sell_days.append(sellday)
            holding_period = sellday - buyday
            holding_period_list.append(holding_period.days)
            print('sell in {} at {} | profit_two is {}'.format(sellday,buy_stopLine,percent))

        #  最終日にまだ持ってたら利確する
        if i == (len(df)-1) and buy_position == True:
            buy_position = False
            percent = (close / buy_price - 1) * 100
            percentChange.append(percent)
            sellday = pd.to_datetime(df.index[i])
            sell_days.append(sellday)
            holding_period = sellday - buyday
            holding_period_list.append(holding_period.days)
            print('Date:'+str(i),'Last Day Buy at the price {}'.format(close))

       
    # statistic
#     ------------------------------------------
    gains = 0
    losses = 0
    numlosses = 0
    numgains = 0
    total_return = 1


    for i in percentChange:
        if i > 0:
            numgains += 1
            gains += i
        else:
            numlosses += 1
            losses += i
        total_return = total_return * ((i / 100) + 1)
        
        
    total_return = round((total_return - 1)*100, 2)
    if numgains > 0:
        ave_gain = gains / numgains
#         max_return = max(percentChange)
    else:
        ave_gain = 0
        max_return = 'unknown'
    
    if numlosses > 0:
        ave_loss = losses / numlosses
#         max_loss = min(percentChange)
        risk_reward_retio = - ave_gain / ave_loss
    else:
        ave_loss = 0
        max_loss = 'unknown'
        risk_reward_retio = 'inf'
    
    if numgains > 0 or numlosses > 0:
        batting_retio = numgains / (numgains + numlosses)
    else:
        batting_retio = 0
    
    trades = numgains + numlosses
    ave_total = ave_gain * batting_retio + ave_loss * (1 - batting_retio)
    
    #average holding period
    if len(holding_period_list) != 0:   
        average_holding_period = sum(holding_period_list)/len(holding_period_list)
    

    results = [numgains+numlosses, round(total_return,2), round(ave_gain,2), round(ave_loss,2)
               ,batting_retio ,ave_total ,average_holding_period]
    
    if not results is None:
         totalresult.append(results)
            


    #plot figure --------------------------------------------------------- 
    
    signal_signals = []
    for i in df.index:
        if i in signal_days:
            signal_signals.append(df['Adj Close'][i])
        else:
            signal_signals.append(np.nan)
    signal_df = pd.DataFrame({'signal_signals':signal_signals})

    
    
    
    x = np.arange(len(df.index)) #x軸を0からデータの個数まで1ずつ増える数列
    labels = df.index #x軸ラベルを日付にする

    
    data = go.Candlestick(x = df.index, open=df['Open'],high=df['High'],low=df['Low'],close=df['Close'],
                         yaxis = 'y1', name  = stock_code)
    sma_short = go.Scatter(x = df.index, y = df['SMA'+str(short_sma)], mode = 'lines',yaxis = 'y1')
    sma_long = go.Scatter(x = df.index, y = df['SMA'+str(long_sma)], mode = 'lines',yaxis = 'y1')
    sma_verylong = go.Scatter(x = df.index, y = df['SMA'+str(verylong_sma)], mode = 'lines',yaxis = 'y1')
    upper_Bband = go.Scatter(x = df.index, y = df['Upper'], mode = 'lines',yaxis = 'y1')
    lower_Bband = go.Scatter(x = df.index, y = df['Lower'], mode = 'lines',yaxis = 'y1')
    Standard_deviation_normalization = go.Scatter(x = df.index, y = df['Standard_deviation_normalization'], 
                                                  mode = 'lines',yaxis = 'y2')
    STD = go.Scatter(x = df.index, y = df['STD'], mode = 'lines',yaxis = 'y3')
    #signal = go.Scatter(x = df.index, y = signal_df , mode = 'scatter',yaxis = 'y1')
     
        
    
        
    
    # レイアウトの指定
    layout = go.Layout(xaxis = dict(title = 'date', type='date', dtick = 'M1'),
                       yaxis = dict(title = 'value') ,
                       yaxis2 = dict(title = 'STD'))  
    # dtick: 'M1'で１ヶ月ごとにラベル表示 
    
    
    
    fig = tools.make_subplots(rows=3, cols=1,
                              subplot_titles=(stock_code,"STD"))
    
    fig.append_trace(data, 1, 1)
    fig.append_trace(sma_short, 1, 1)
    fig.append_trace(sma_long, 1, 1)
    fig.append_trace(sma_long, 1, 1)
    fig.append_trace(Standard_deviation_normalization, 2, 1)
    fig.append_trace(STD, 3, 1)
    
    
    fig['layout'].update(height=1300, width=1200)

    fig.show()
    
#      = go.Scatter(x = df.index, y = df['SMA'+str(short_sma)], mode = 'lines')

    
# #     make figure
# #     df['70'] = 70
# #     df['30'] = 30
# #     df['50'] = 50
# #     df['0'] = 0
    
# #     signal_signals = []
# #     for i in df.index:
# #         if i in signal_days:
# #             signal_signals.append(df['Adj Close'][i])
# #         else:
# #             signal_signals.append(np.nan)
            
            
# #     if np.all(signal_signals) == True:
# #         apds = [
# #         mpl.make_addplot( df['Volume'] , panel=3, ylabel='Volume'),
# #         mpl.make_addplot(df['Upper'], type='line' ,color="r"),
# #         mpl.make_addplot(df['Lower'], type='line',color="r"),
# #         mpl.make_addplot(df['Standard_deviation_normalization'], panel=1, type='line', ylabel='STD'),
# #         mpl.make_addplot( df[['daviation_rate','0']] , panel=2, type='line', ylabel='daviation_rate'),
# #         ]
   
# # # mpl.make_addplot(signal_signals, type='scatter', markersize=10000, marker='|', color="b"),
# #         fig = mpl.plot(df, type='candle',figsize=(30, 15), style='yahoo',mav=(short_sma, long_sma, verylong_sma)
# #                        , addplot=apds, title=str(stock_code))



columns = ['trades', 'Total return', 'Average Gain', 'Average Loss', '勝率' ,'Average gain and profit','average holding period']
df = pd.DataFrame(totalresult, columns=columns, index = sign)
df.to_csv('./volatility_result2.csv')

print(df)
print(1)

[*********************100%***********************]  1 of 1 completed
6976.T
              Open    High     Low   Close    Adj Close   Volume  Highest81
Date                                                                       
2019-01-04  1537.0  1537.0  1407.0  1476.0  1448.142334  6397500        NaN
2019-01-07  1535.0  1572.0  1510.0  1541.0  1511.915405  3628300        NaN
2019-01-08  1567.0  1652.0  1561.0  1637.0  1606.103516  4389000        NaN
2019-01-09  1574.0  1581.0  1512.0  1564.0  1534.481323  7343100        NaN
2019-01-10  1543.0  1571.0  1497.0  1509.0  1480.519409  4917100        NaN
2019-03-08 00:00:00
2019-03-12 00:00:00
2019-03-27 00:00:00
2019-03-28 00:00:00
2019-03-29 00:00:00
2019-04-01 00:00:00
2019-04-02 00:00:00
2019-06-11 00:00:00
2019-06-19 00:00:00
2019-06-20 00:00:00
2019-06-21 00:00:00
2019-06-24 00:00:00
2019-06-25 00:00:00
2019-06-26 00:00:00
2019-06-27 00:00:00
2019-06-28 00:00:00
2019-07-29 00:00:00
2019-07-30 00:00:00
2019-07-31 00:00:00
2019-08-02 0

C:\Users\remuo402\anaconda3\envs\stockTrade\lib\site-packages\plotly\tools.py:461: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



[*********************100%***********************]  1 of 1 completed
4477.T
              Open    High     Low   Close  Adj Close   Volume  Highest81
Date                                                                     
2019-10-25  1210.0  1410.0  1210.0  1333.0     1333.0  6088700        NaN
2019-10-28  1310.0  1324.0  1281.0  1312.0     1312.0   651900        NaN
2019-10-29  1308.0  1345.0  1177.0  1200.0     1200.0   750200        NaN
2019-10-30  1170.0  1230.0  1169.0  1200.0     1200.0   291000        NaN
2019-10-31  1192.0  1240.0  1191.0  1235.0     1235.0   121200        NaN
2019-12-23 00:00:00
2020-01-07 00:00:00
2020-01-08 00:00:00
2020-01-16 00:00:00
2020-01-17 00:00:00
2020-01-20 00:00:00
2020-01-29 00:00:00
buy in 2020-05-11 00:00:00 at 1975.0
sell in 2020-05-13 00:00:00 at 1851.0 | profit_one is -6.278481012658233


C:\Users\remuo402\anaconda3\envs\stockTrade\lib\site-packages\plotly\tools.py:461: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



[*********************100%***********************]  1 of 1 completed
6036.T
             Open   High    Low  Close   Adj Close  Volume  Highest81
Date                                                                 
2019-01-04  445.0  449.5  443.0  445.0  436.407166   41800        NaN
2019-01-07  455.0  464.5  451.0  464.5  455.530609   22400        NaN
2019-01-08  464.5  477.5  464.5  477.5  468.279572   19200        NaN
2019-01-09  492.5  492.5  478.5  486.0  476.615417   29200        NaN
2019-01-10  482.5  483.0  468.5  478.0  468.769928   30400        NaN
2019-02-08 00:00:00
2019-03-08 00:00:00
2019-03-20 00:00:00
2019-03-25 00:00:00
2019-04-18 00:00:00
2019-04-19 00:00:00
2019-04-22 00:00:00
2019-04-23 00:00:00
2019-04-24 00:00:00
2019-04-25 00:00:00
2019-04-26 00:00:00
2019-05-07 00:00:00
2019-05-08 00:00:00
2019-05-15 00:00:00
2019-05-16 00:00:00
2019-05-17 00:00:00
2019-05-20 00:00:00
2019-05-21 00:00:00
2019-05-22 00:00:00
2019-05-23 00:00:00
buy in 2019-05-24 00:00:00 at 709.

C:\Users\remuo402\anaconda3\envs\stockTrade\lib\site-packages\plotly\tools.py:461: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



        trades  Total return  Average Gain  Average Loss   勝率  \
6976.T       4         83.39         46.17         -6.00  0.5   
4477.T       1         -6.28          0.00         -6.28  0.0   
6036.T       6        104.24         45.96         -6.61  0.5   

        Average gain and profit  average holding period  
6976.T                20.085531               45.333333  
4477.T                -6.278481               45.333333  
6036.T                19.673998               49.750000  
1
